<h1><center>Billboard's Latin Streaming Songs </center></h1>
<h2><center>Year-end and Weekly charts (2013 - 2020) </center></h2>
<h3><center>Data downloading and database cleaning notebook </center></h3>

## I. *Year-end charts* de 2013 a 2020

### I.1 Descarga de los datos.

Se seguirán prácticamente los mismos pasos que en la sección 2 del notebook 'Prueba Billboard'

In [2]:
import pandas as pd
import numpy as np
import billboard
import itertools
import os

In [2]:
#Descargamos todos los year-end charts de 'latin-streaming-songs' desde 2013.
years = list(range(2013, 2021, 1))

#Los años serán las keys de los diccionarios
year_end_dict = {}
for y in years:
    year_end_dict[y] = billboard.ChartData('latin-streaming-songs', year = y)
    
#Accedemos a cualquier chart con 'year_end_dict[año]' y a cualquier posición con 'year_end_dict[año][poisicion de 0 a 24]'

### I.2 Limpieza y preparación de los datos.

In [3]:
#Ahora convertimos cada elemento del diccionario anidado 'year_end' en un solo DataFrame.

dicts = [year_end_dict[x] for x in years] #Extraemos cada chart del diccionario anidado a una lista.
dicts = [pd.DataFrame.from_records([{'chart': dicts[x]}]) for x in range(len(years))] #Lista a 8 dfs.

year_end = pd.concat(dicts, ignore_index=True) #Concatenamos (append) los 8 dfs para tener uno solo.
year_end

,chart
0,"('Danza Kuduro' by Don Omar & Lucenzo, 'Waka W..."
1,('Bailando' by Enrique Iglesias Featuring Desc...
2,('Bailando' by Enrique Iglesias Featuring Desc...
3,('Bailando' by Enrique Iglesias Featuring Desc...
4,('Despacito' by Luis Fonsi & Daddy Yankee Feat...
5,('Despacito' by Luis Fonsi & Daddy Yankee Feat...
6,('Taki Taki' by DJ Snake Featuring Selena Gome...
7,"('Vete' by Bad Bunny, 'Tusa' by Karol G & Nick..."


In [4]:
#Agregamos al df recién creado los años de cada chart
year_end['year'] = years
year_end = year_end.set_index('year') #cada fila es un year-end chart.
year_end

,chart
year,
2013,"('Danza Kuduro' by Don Omar & Lucenzo, 'Waka W..."
2014,('Bailando' by Enrique Iglesias Featuring Desc...
2015,('Bailando' by Enrique Iglesias Featuring Desc...
2016,('Bailando' by Enrique Iglesias Featuring Desc...
2017,('Despacito' by Luis Fonsi & Daddy Yankee Feat...
2018,('Despacito' by Luis Fonsi & Daddy Yankee Feat...
2019,('Taki Taki' by DJ Snake Featuring Selena Gome...
2020,"('Vete' by Bad Bunny, 'Tusa' by Karol G & Nick..."


In [10]:
#Creamos una funciones para extraer todos los atributos de la canciones en los charts. 

def get_entry(df, chart, position, att = ''):
    if att == 'artist':
        return [df['chart'].iat[chart][position].artist]
    elif att == 'title':
        return [df['chart'].iat[chart][position].title]
    elif att == 'rank':
        return [df['chart'].iat[chart][position].rank]
    elif att == 'peakPos':
        return [df['chart'].iat[chart][position].peakPos]
    elif att == 'weeks':
        return [df['chart'].iat[chart][position].weeks]
    elif att == 'lastPos':
        return [df['chart'].iat[chart][position].lastPos]
    elif att == 'image':
        return [df['chart'].iat[chart][position].image]
    else:
        return print('Select one entry, please')
    

In [6]:
#Good to know 2
#print(range(len(years)))
#print(range(len(year_end['chart'].iloc[0])))
#print(year_end['chart'].iloc[7])

In [7]:
#Después de crear las funciones, cada extracción que arroja la función las ponemos en una lista

artists = []
for y in range(len(years)):
    for j in range(len(year_end['chart'].iloc[y])):
        artists.append(get_entry(year_end, y, j, 'artist'))
        
titles = []
for y in range(len(years)):
    for j in range(len(year_end['chart'].iloc[y])):
        titles.append(get_entry(year_end, y, j, 'title'))
        
ranking = []
for y in range(len(years)):
    for j in range(len(year_end['chart'].iloc[y])):
        ranking.append(get_entry(year_end, y, j, 'rank'))


In [8]:
#Ahora unimos las listas en un dataframe
year_end = pd.DataFrame({'Rank': ranking, 'Title': titles, 'Artist': artists})
year_end

,Rank,Title,Artist
0,[1],[Danza Kuduro],[Don Omar & Lucenzo]
1,[2],[Waka Waka (Esto Es Africa)],[Shakira Featuring Freshlyground]
2,[3],[Hips Don't Lie],[Shakira Featuring Wyclef Jean]
3,[4],[Darte Un Beso],[Prince Royce]
4,[5],[Vivir Mi Vida],[Marc Anthony]
...,...,...,...
269,[46],[La Curiosidad],[DJ Nelson Presenta Jay Wheeler & Myke Towers]
270,[47],[Calma],[Pedro Capo X Farruko]
271,[48],[La Mejor Version de Mi],[Natti Natasha & Romeo Santos]
272,[49],[Morado],[J Balvin]


In [9]:
#Como cada observación del df es una lista, debemos sacarla de ella.
#Esta parte de código no es tan automática, pero sirve alv (gracias a StackOverflow)
#Hacemos una variable por una.

#Artista
rows = []
artist = year_end.apply(lambda row: [rows.append([row['Rank'], row['Title'], Artist]) 
                                         for Artist in row.Artist], axis = 1)
df_new = pd.DataFrame(rows, columns=year_end.columns)

#Title
rows2 = []
title = df_new.apply(lambda row: [rows2.append([row['Rank'], Title, row['Artist']]) 
                                         for Title in row.Title], axis = 1)
df_new2 = pd.DataFrame(rows2, columns=df_new.columns)


#Rank
rows3 = []
rank = df_new2.apply(lambda row: [rows3.append([Rank, row['Title'], row['Artist']]) 
                                         for Rank in row.Rank], axis = 1)
year_end = pd.DataFrame(rows3, columns=df_new2.columns)

year_end.head(5)

,Rank,Title,Artist
0,1,Danza Kuduro,Don Omar & Lucenzo
1,2,Waka Waka (Esto Es Africa),Shakira Featuring Freshlyground
2,3,Hips Don't Lie,Shakira Featuring Wyclef Jean
3,4,Darte Un Beso,Prince Royce
4,5,Vivir Mi Vida,Marc Anthony


In [10]:
#Liberamos memoria
df_to_drop = [artist, title, rank, df_new, df_new2]
del df_to_drop

Ahora, ya que tenemos el dataframe con todas los charts, agregamos los años y limpiamos para obtener cada artista separado por comas. Despúes de eso creamos dos variables extras: el artista principal (*source*) y los colaboradores (*target*)

In [11]:
##                                Agregamos variable de Año

#Función que me regresa una serie que contiene el año 'year' repetido 'i' veces.
def year_times(x, year):
    i = len(x[year]) #extrae número de canciones del chart en año 'year'
    S = pd.Series(year) #guarda el año en un objeto: Series
    S = S.repeat(i) #repite el año tantas canciones hayan en el chart
    return S 

#Cremos una lista de todos los años trabajados con ayuda de la función 'year_times()' para agregarla al dataframe
year_to_df = []
for y in years:
    year_to_df.append(year_times(year_end_dict, y))

#El loop anterior nos da una lista con 'year' listas dentro, por lo tanto hay que hacer sólo una lista
year_to_df = list(itertools.chain.from_iterable(year_to_df))


#Agregamos la lista de años recién creada al dataframe y reordenamos columnas
year_end['Year'] = year_to_df

column_names = ['Year', 'Rank', 'Title', 'Artist']
year_end = year_end.reindex(columns = column_names)
year_end

,Year,Rank,Title,Artist
0,2013,1,Danza Kuduro,Don Omar & Lucenzo
1,2013,2,Waka Waka (Esto Es Africa),Shakira Featuring Freshlyground
2,2013,3,Hips Don't Lie,Shakira Featuring Wyclef Jean
3,2013,4,Darte Un Beso,Prince Royce
4,2013,5,Vivir Mi Vida,Marc Anthony
...,...,...,...,...
269,2020,46,La Curiosidad,DJ Nelson Presenta Jay Wheeler & Myke Towers
270,2020,47,Calma,Pedro Capo X Farruko
271,2020,48,La Mejor Version de Mi,Natti Natasha & Romeo Santos
272,2020,49,Morado,J Balvin


In [12]:
#Observamos qué expresiones separan a los artistas en cada canción
year_end['Artist'].unique()

array(['Don Omar & Lucenzo', 'Shakira Featuring Freshlyground',
       'Shakira Featuring Wyclef Jean', 'Prince Royce', 'Marc Anthony',
       'Michel Telo', 'Romeo Santos', 'Roberto Junior y Su Bandeno',
       'Shakira', 'Gerardo Ortiz',
       'Enrique Iglesias Featuring Romeo Santos',
       'Romeo Santos Featuring Usher', 'Daddy Yankee',
       'Wisin & Yandel Featuring Chris Brown & T-Pain',
       'Shakira Featuring El Cata', 'J Alvarez',
       'Alejandro Fernandez / Christina Aguilera', 'Aventura', 'Arcangel',
       'Romeo Santos, Anthony "El Mayimbe" Santos, Luis Varges & Raulin Rodriguez',
       'Enrique Iglesias Featuring Descemer Bueno & Gente de Zona',
       'Romeo Santos Featuring Drake',
       'Enrique Iglesias Featuring Marco Antonio Solis',
       'Banda Sinaloense MS de Sergio Lizarraga',
       'Wisin Featuring Jennifer Lopez & Ricky Martin',
       'Banda Tierra Sagrada', 'El Komander',
       'Nicky Jam & Enrique Iglesias', 'J Balvin Featuring Farruko',
      

In [13]:
#Como todos los artistas están separados de diferentes maneras, estandarizamos la separación con comas (,)

#Estandarizamos algunos nombres de artistas.
year_end['Artist'] = year_end['Artist'].str.replace('Jowell & Randy', 'Jowell-Randy')
year_end['Artist'] = year_end['Artist'].str.replace('J. Balvin', 'J Balvin')
year_end['Artist'] = year_end['Artist'].str.replace('Zion & Lennox', 'Zion-Lennox')
year_end['Artist'] = year_end['Artist'].str.replace('Wisin & Yandel', 'Wisin-Yandel')
year_end['Artist'] = year_end['Artist'].str.replace('Banda Los Sebastianes de Mazatlan, Sinaloa.', 
                                                    'Banda Los Sebastianes de Mazatlan Sinaloa')

#Una vez estandarizados los nombres, reemplazamos los strings separadores de artistas por comas.
strings = [' &', ' X', ' x', ' \+', '\/', ' Or ', ' Featuring', ' Presenta', 'Presentan:', 'n\:']
for st in strings:
    year_end['Artist'] = year_end['Artist'].str.replace(st, ', ')


#Comprobamos.
year_end['Artist'].unique()

array(['Don Omar,  Lucenzo', 'Shakira,  Freshlyground',
       'Shakira,  Wyclef Jean', 'Prince Royce', 'Marc Anthony',
       'Michel Telo', 'Romeo Santos', 'Roberto Junior y Su Bandeno',
       'Shakira', 'Gerardo Ortiz', 'Enrique Iglesias,  Romeo Santos',
       'Romeo Santos,  Usher', 'Daddy Yankee',
       'Wisin-Yandel,  Chris Brown,  T-Pain', 'Shakira,  El Cata',
       'J Alvarez', 'Alejandro Fernandez ,  Christina Aguilera',
       'Aventura', 'Arcangel',
       'Romeo Santos, Anthony "El Mayimbe" Santos, Luis Varges,  Raulin Rodriguez',
       'Enrique Iglesias,  Descemer Bueno,  Gente de Zona',
       'Romeo Santos,  Drake', 'Enrique Iglesias,  Marco Antonio Solis',
       'Banda Sinaloense MS de Sergio Lizarraga',
       'Wisin,  Jennifer Lopez,  Ricky Martin', 'Banda Tierra Sagrada',
       'El Komander', 'Nicky Jam,  Enrique Iglesias',
       'J Balvin,  Farruko', 'J Balvin', 'Nicky Jam', 'Plan B',
       'Ariel Camacho y Los Plebes del Rancho',
       'Enrique Iglesias, 

In [14]:
#Ahora separamos el artista principal (source) de los colaboradores (target).
year_end[['Principal','Collaborator']] = year_end['Artist'].str.split(',', 1, expand=True)

#Eliminamos la columna 'Artist'.
year_end = year_end.drop('Artist', axis = 1)

#Reemplazamos valores 'None' por 'Na'.
year_end.fillna('', inplace = True)

#Vemos las primeras 5 observaciones
year_end.head()

,Year,Rank,Title,Principal,Collaborator
0,2013,1,Danza Kuduro,Don Omar,Lucenzo
1,2013,2,Waka Waka (Esto Es Africa),Shakira,Freshlyground
2,2013,3,Hips Don't Lie,Shakira,Wyclef Jean
3,2013,4,Darte Un Beso,Prince Royce,
4,2013,5,Vivir Mi Vida,Marc Anthony,


In [15]:
#Hacemos explode de la variable 'Collaborator' para obtener un artista por renglón.
year_end = year_end.assign(Collaborator = year_end['Collaborator'].str.split(', ')).explode('Collaborator')
year_end = year_end.reset_index(drop = True)

In [16]:
#Removemos espacios en blanco que hayan quedado al principio de cada artista en la variable 'Collaborator'.
year_end['Collaborator'] = year_end['Collaborator'].str.lstrip()

In [17]:
#Vemoss las primeras 10 observaciones.
year_end

,Year,Rank,Title,Principal,Collaborator
0,2013,1,Danza Kuduro,Don Omar,Lucenzo
1,2013,2,Waka Waka (Esto Es Africa),Shakira,Freshlyground
2,2013,3,Hips Don't Lie,Shakira,Wyclef Jean
3,2013,4,Darte Un Beso,Prince Royce,
4,2013,5,Vivir Mi Vida,Marc Anthony,
...,...,...,...,...,...
385,2020,46,La Curiosidad,DJ Nelson,Myke Towers
386,2020,47,Calma,Pedro Capo,Farruko
387,2020,48,La Mejor Version de Mi,Natti Natasha,Romeo Santos
388,2020,49,Morado,J Balvin,


### I.3 Exportamos a un csv

In [22]:
path_data = r'C:\Users\Francisco Martínez\Desktop\Economía de redes\Trabajo final\Data'
os.chdir(path_data)

# year_end.to_csv('year_end_charts.csv', index = False)

## II. *Weekly charts* de 2013 a 2020

### II.1 Descarga de los datos

La descarga será un poco diferente, pues primero hay que encontrar todos los sábados (día en que Billboard publica los charts) de 2013 a 2020. Por lo tanto, primero hacemos uso de los módulos `datetime` y `calendar`.

In [3]:
import datetime
import calendar

In [4]:
#Función que regrese cualquier sábado de cualquier año con formato 'YYYY-MM-DD'

def saturday(year, month, week):
    calendar.setfirstweekday(calendar.SUNDAY)
    c = np.array(calendar.monthcalendar(year, month))
    c = c[week - 1, 6]
    if month in range(1, 10, 1):
        date = '{}-0{}-{}'.format(year, month, c)
    elif c in range(1, 10, 1):
        date = '{}-{}-0{}'.format(year, month, c)
    else:
        date = '{}-{}-{}'.format(year, month, c)
    return date


#Función que retorne el número de semanas (sábados en este caso) entre una fecha  y otra

def no_weeks(date1, date2): #El input son fechas obtenidas con la función 'saturday()'
    #Separamos los strings
    d1 = date1
    d1 = d1.split('-')
    d2 = date2
    d2 = d2.split('-')
    #Usamos la función date() para conocer las fechas y hacer la operación deseada
    d1 = datetime.date(int(d1[0]), int(d1[1]), int(d1[2]))
    d2 = datetime.date(int(d2[0]), int(d2[1]), int(d2[2]))
    #Conocemos primero los días entre ambas fechas
    days = abs(d2 - d1).days
    #Dividimos entre 7 para conocer las semanas
    weeks = int(days / 7)
    return weeks        

In [5]:
#Buscamos todos los sábados de 2013 a 2020 y los convertimos en un dataframe.

#Cada weekly chart de Billboard tiene diferentes fechas de inicio. 
#Para el caso del 'weekly-latin-streaming' inicia el 20 de abril de 2013.
sat1 = saturday(2013, 4, 3)
satn = saturday(2020, 12, 4)

sats = pd.date_range(sat1, periods = (no_weeks(sat1, satn) + 1), freq = "W-SAT")
sats = pd.DataFrame({'date': sats})
sats.iat[0,0]

#La fecha incluye las horas, minutos y segundos, así que los eliminamos para únicamente tener un formato: YYYY-MM-DD.
sats['date'] = sats['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
sats.iat[0,0]

'2013-04-20'

In [6]:
#Descargamos los charts semanales desde Billboard.
#Creamos un diccionario con cada chart semanal con las fechas que obtuvimos anteriormente.
w_charts = {} #dict vacío.
for s in range(len(sats['date'])):
    w_charts[s] = billboard.ChartData('latin-streaming-songs', date = sats.iat[s,0])

### II.2 Limpieza y preparación de los datos.

In [7]:
#Cambiamos las keys del diccionario por la fecha. Será útil después.
#Incluimos el mes y día porque las keys del diccionario no se pueden duplicar

#De 'sats' extraemos la fecha y le aplicamos la función 'split()' y después la reunimos, pero sin separadores.
key = pd.DataFrame({'date': sats['date']})
key[['y', 'm', 'd']] = key['date'].str.split('-', expand = True)
key['ymd'] = key['y'] + key['m'] + key['d']

#Sobreescribimos 'key' en una lista con los valores de 'key['ymd']'.
key = list(key['ymd'])

#Extraemos los valores de 'w_charts' y los guardamos en una lista.
value = list(w_charts.values())

#Ya que tenemos las listas con las 'keys' y los 'values', creamos una función para volverlas una lista de tuples.
def list_of_tup(list1, list2):
    t = [((list1[0]), (list2[0]))]
    for i in range(1, len(list2)):
        t.append(((list1[i]), (list2[i])))
    return t

#Creamos la lista de tuples.
tuplelist = list_of_tup(key, value)

#Cambiamos, con la lista de tuples, los valores de las 'keys' por las fechas sin separadores.
w_charts = dict((x, index) for x, index in tuplelist)    

In [8]:
#Ahora convertimos cada elemento del diccionario anidado en un solo DataFrame.

dicts = [w_charts[x] for x in key] #Extraemos cada chart semanal del diccionario anidado a un array.
dicts = [pd.DataFrame.from_records([{'chart': dicts[x]}]) for x in range(len(key))] #Convertimos el array a x dfs.

weekly = pd.concat(dicts, ignore_index=True) #Concatenamos (append) los 52 dts para tener uno solo.
weekly

,chart
0,('Hips Don't Lie' by Shakira Featuring Wyclef ...
1,('Hips Don't Lie' by Shakira Featuring Wyclef ...
2,('Waka Waka (Esto Es Africa)' by Shakira Featu...
3,('Waka Waka (Esto Es Africa)' by Shakira Featu...
4,('Waka Waka (Esto Es Africa)' by Shakira Featu...
...,...
397,"('Dakiti' by Bad Bunny & Jhay Cortez, 'Hawai' ..."
398,"('Dakiti' by Bad Bunny & Jhay Cortez, 'Hawai' ..."
399,"('Dakiti' by Bad Bunny & Jhay Cortez, 'Feliz N..."
400,"('Dakiti' by Bad Bunny & Jhay Cortez, 'Feliz N..."


In [11]:
#Extraemmos los atributos de cada chart semanal con la ayuda de la función 'get_entry()' que definimos anteriormente

#Atributos a extraer: artista, titulo, peak position, posición en el ranking y semanas en chart.

artists = []
for i in range(len(weekly['chart'])):
    for j in range(len(weekly['chart'].iloc[i])):
        artists.append(get_entry(weekly, i, j, 'artist'))
        
titles = []
for i in range(len(weekly['chart'])):
    for j in range(len(weekly['chart'].iloc[i])):
        titles.append(get_entry(weekly, i, j, 'title'))
         
peak = []
for i in range(len(weekly['chart'])):
    for j in range(len(weekly['chart'].iloc[i])):
        peak.append(get_entry(weekly, i, j, 'peakPos'))

week = []
for i in range(len(weekly['chart'])):
    for j in range(len(weekly['chart'].iloc[i])):
        week.append(get_entry(weekly, i, j, 'weeks'))

ranking = []
for i in range(len(weekly['chart'])):
    for j in range(len(weekly['chart'].iloc[i])):
        ranking.append(get_entry(weekly, i, j, 'rank'))

In [12]:
#Ahora unimos las listas en un dataframe
weekly = pd.DataFrame({'Rank': ranking, 'Title': titles, 'Artist': artists, 'Peak_position': peak, 
                             'Weeks_in_chart': week})
weekly

,Rank,Title,Artist,Peak_position,Weeks_in_chart
0,[1],[Hips Don't Lie],[Shakira Featuring Wyclef Jean],[1],[1]
1,[2],[Waka Waka (Esto Es Africa)],[Shakira Featuring Freshlyground],[2],[1]
2,[3],[Danza Kuduro],[Don Omar & Lucenzo],[3],[1]
3,[4],[Ai Se Eu Te Pego],[Michel Telo],[4],[1]
4,[5],[Loca],[Shakira Featuring El Cata],[5],[1]
...,...,...,...,...,...
10045,[21],[Callaita],[Bad Bunny & Tainy],[1],[79]
10046,[22],[La Toxica],[Farruko],[6],[8]
10047,[23],[Si Veo A Tu Mama],[Bad Bunny],[1],[40]
10048,[24],[Maldita Pobreza],[Bad Bunny],[12],[3]


In [13]:
#Como cada observación del df es una lista, debemos sacarla de ella.
#Esta parte de código no es tan automática, pero sirve alv (gracias a StackOverflow)
#Hacemos una variable por una.

#Artista
rows = []
prueba = weekly.apply(lambda row: [rows.append([row['Rank'], row['Title'], Artist, 
                                                      row['Peak_position'], row['Weeks_in_chart']]) 
                                         for Artist in row.Artist], axis = 1)
df_new = pd.DataFrame(rows, columns=weekly.columns)
df_new

#Title
rows2 = []
prueba2 = df_new.apply(lambda row: [rows2.append([row['Rank'], Title, row['Artist'], 
                                                      row['Peak_position'], row['Weeks_in_chart']]) 
                                         for Title in row.Title], axis = 1)
df_new2 = pd.DataFrame(rows2, columns=df_new.columns)
df_new2

#Rank
rows3 = []
prueba3 = df_new2.apply(lambda row: [rows3.append([Rank, row['Title'], row['Artist'], 
                                                      row['Peak_position'], row['Weeks_in_chart']]) 
                                         for Rank in row.Rank], axis = 1)
df_new3 = pd.DataFrame(rows3, columns=df_new2.columns)
df_new3

#Peak Position
rows4 = []
prueba4 = df_new3.apply(lambda row: [rows4.append([row['Rank'], row['Title'], row['Artist'], 
                                                   Peak_position, row['Weeks_in_chart']]) 
                                     for Peak_position in row.Peak_position], axis = 1)
df_new4 = pd.DataFrame(rows4, columns=df_new3.columns)
df_new4


#Weeks in chart
rows5 = []
prueba5 = df_new4.apply(lambda row: [rows5.append([row['Rank'], row['Title'], row['Artist'], 
                                                   row['Peak_position'], Weeks_in_chart]) 
                                     for Weeks_in_chart in row.Weeks_in_chart], axis = 1)
weekly = pd.DataFrame(rows5, columns=df_new4.columns)
weekly

,Rank,Title,Artist,Peak_position,Weeks_in_chart
0,1,Hips Don't Lie,Shakira Featuring Wyclef Jean,1,1
1,2,Waka Waka (Esto Es Africa),Shakira Featuring Freshlyground,2,1
2,3,Danza Kuduro,Don Omar & Lucenzo,3,1
3,4,Ai Se Eu Te Pego,Michel Telo,4,1
4,5,Loca,Shakira Featuring El Cata,5,1
...,...,...,...,...,...
10045,21,Callaita,Bad Bunny & Tainy,1,79
10046,22,La Toxica,Farruko,6,8
10047,23,Si Veo A Tu Mama,Bad Bunny,1,40
10048,24,Maldita Pobreza,Bad Bunny,12,3


In [14]:
#                                   Agregamos variable de año

#NO Haremos uso de la función 'year_times()' creada anteriormente porque por la estructura del diccionario, no nos sirve
#Creamos otra función parecida.

def date_times(x, saturday_date): #Formato de la fecha del sábado: YYYY-MM-DD.
    S = saturday_date
    S = S.split('-') #Separa la fecha y la guarda en una lista: ['YYYY', 'MM', 'DD'].
    S = S[0] + S[1] + S[2] #Junta la fecha para que quede 'YYYYMMDD'.
    i = len(x[S]) #Extrae el número de canciones de la fecha del input.
    S = pd.Series(S) #Convierte la fecha en una Series.
    S = S.repeat(i) #repite la fecha tantas canciones hayan en el chart.
    return S

In [15]:
#Cremos una lista de todas las fechas.
saturday_dates = list(sats['date'])

date_to_df = []
for d in saturday_dates:
    date_to_df.append(date_times(w_charts, d))

#El loop anterior nos da una lista con 'date' listas dentro, por lo tanto hay que hacer sólo una lista
date_to_df = list(itertools.chain.from_iterable(date_to_df))


#Agregamos la lista de años recién creada al dataframe y reordenamos columnas
weekly['Date'] = date_to_df

column_names = ['Date', 'Rank', 'Title', 'Artist', 'Peak_position', 'Weeks_in_chart']
weekly = weekly.reindex(columns = column_names)
weekly

,Date,Rank,Title,Artist,Peak_position,Weeks_in_chart
0,20130420,1,Hips Don't Lie,Shakira Featuring Wyclef Jean,1,1
1,20130420,2,Waka Waka (Esto Es Africa),Shakira Featuring Freshlyground,2,1
2,20130420,3,Danza Kuduro,Don Omar & Lucenzo,3,1
3,20130420,4,Ai Se Eu Te Pego,Michel Telo,4,1
4,20130420,5,Loca,Shakira Featuring El Cata,5,1
...,...,...,...,...,...,...
10045,20201226,21,Callaita,Bad Bunny & Tainy,1,79
10046,20201226,22,La Toxica,Farruko,6,8
10047,20201226,23,Si Veo A Tu Mama,Bad Bunny,1,40
10048,20201226,24,Maldita Pobreza,Bad Bunny,12,3


In [16]:
#Ahora separamos 'Date' en 'Year', 'Month' y 'Day'.

#Para hacerlo creamos una función (para este punto estoy lo suf cansado como para pelear con regular expressions).
def extract_date_entry(df, obs_no, entry = ''):
    date = df['Date'].iat[obs_no]
    if entry == 'year':
        return date[0:4]
    elif entry == 'month':
        return date[4:6]
    elif entry == 'day':
        return date[6:8]
    else:
        return print('Select date entry: either year, month, or day.')
    

#Usamos la función recién creada para crear 3 listas: Year, Month, Day.
Year = []
for i in range(len(weekly)):
    Year.append(extract_date_entry(weekly, i, 'year'))

Month = []
for i in range(len(weekly)):
    Month.append(extract_date_entry(weekly, i, 'month'))
    
Day = []
for i in range(len(weekly)):
    Day.append(extract_date_entry(weekly, i, 'day'))

#Agregamos las listas recién creadas al df y eliminamos la variable 'Date'.
weekly['Year'] = Year
weekly['Month'] = Month
weekly['Day'] = Day

weekly = weekly.drop('Date', axis = 1)

#Reordenamos columnas.
column_names = ['Year', 'Month', 'Day', 'Rank', 'Title', 'Artist', 'Peak_position', 'Weeks_in_chart']
weekly = weekly.reindex(columns = column_names)
weekly

,Year,Month,Day,Rank,Title,Artist,Peak_position,Weeks_in_chart
0,2013,04,20,1,Hips Don't Lie,Shakira Featuring Wyclef Jean,1,1
1,2013,04,20,2,Waka Waka (Esto Es Africa),Shakira Featuring Freshlyground,2,1
2,2013,04,20,3,Danza Kuduro,Don Omar & Lucenzo,3,1
3,2013,04,20,4,Ai Se Eu Te Pego,Michel Telo,4,1
4,2013,04,20,5,Loca,Shakira Featuring El Cata,5,1
...,...,...,...,...,...,...,...,...
10045,2020,12,26,21,Callaita,Bad Bunny & Tainy,1,79
10046,2020,12,26,22,La Toxica,Farruko,6,8
10047,2020,12,26,23,Si Veo A Tu Mama,Bad Bunny,1,40
10048,2020,12,26,24,Maldita Pobreza,Bad Bunny,12,3


In [17]:
#Observamos cómo se encuentran separados y escritos los/las artistas
weekly['Artist'].unique()

array(['Shakira Featuring Wyclef Jean', 'Shakira Featuring Freshlyground',
       'Don Omar & Lucenzo', 'Michel Telo', 'Shakira Featuring El Cata',
       'Shakira', 'Romeo Santos Featuring Usher', 'Belinda',
       'Romeo Santos', 'Wisin & Yandel Featuring Chris Brown & T-Pain',
       'Roberto Junior y Su Bandeno',
       'Shakira Featuring Pitbull Or El Cata', 'Prince Royce',
       'Daddy Yankee', 'Rey Sanchez', 'Arcangel', 'Pitbull',
       'Voz de Mando', 'Gerardo Ortiz', 'Selena', 'Malu',
       'Romeo Santos, Anthony "El Mayimbe" Santos, Luis Varges & Raulin Rodriguez',
       'Maria Jose', 'J Alvarez', 'Zion, Jory y Ken-Y', 'Marc Anthony',
       'Denise de Kalafe', 'Pesado', 'Aventura', 'Loona', 'Luis Coronel',
       'Daddy Yankee Featuring J. Alvarez',
       'Daddy Yankee Featuring Natalia Jimenez', 'Reik',
       'La Arrolladora Banda el Limon de Rene Camacho',
       'Banda Sinaloense MS de Sergio Lizarraga',
       'Alejandro Fernandez / Christina Aguilera', 'Xtreme', '

In [18]:
#Estandarizamos nombres de artistas.
weekly['Artist'] = weekly['Artist'].str.replace('Jowell & Randy', 'Jowell-Randy')
weekly['Artist'] = weekly['Artist'].str.replace('Zion & Lennox', 'Zion-Lennox')
weekly['Artist'] = weekly['Artist'].str.replace('Wisin & Yandel', 'Wisin-Yandel')
weekly['Artist'] = weekly['Artist'].str.replace('Banda Los Sebastianes de Mazatlan, Sinaloa.', 
                                                    'Banda Los Sebastianes de Mazatlan Sinaloa')
weekly['Artist'] = weekly['Artist'].str.replace('Marco Antonio Solis y Los Bukis', 'Marco Antonio Solis, Los Bukis')
weekly['Artist'] = weekly['Artist'].str.replace('J. Balvin', 'J Balvin')
weekly['Artist'] = weekly['Artist'].str.replace('Jory y Ken-Y', 'Jory, Ken-Y')

#Una vez estandarizados los nombres, reemplazamos los strings separadores de artistas por comas.
strings = [' &', ' X', ' x', ' \+', ' Or ', '\/', ' Featuring', ' Presenta', ' Presentan:', ' Duet With', ' n:']
for st in strings:
    weekly['Artist'] = weekly['Artist'].str.replace(st, ', ')


#Comprobamos.
weekly['Artist'].unique()

array(['Shakira,  Wyclef Jean', 'Shakira,  Freshlyground',
       'Don Omar,  Lucenzo', 'Michel Telo', 'Shakira,  El Cata',
       'Shakira', 'Romeo Santos,  Usher', 'Belinda', 'Romeo Santos',
       'Wisin-Yandel,  Chris Brown,  T-Pain',
       'Roberto Junior y Su Bandeno', 'Shakira,  Pitbull, El Cata',
       'Prince Royce', 'Daddy Yankee', 'Rey Sanchez', 'Arcangel',
       'Pitbull', 'Voz de Mando', 'Gerardo Ortiz', 'Selena', 'Malu',
       'Romeo Santos, Anthony "El Mayimbe" Santos, Luis Varges,  Raulin Rodriguez',
       'Maria Jose', 'J Alvarez', 'Zion, Jory, Ken-Y', 'Marc Anthony',
       'Denise de Kalafe', 'Pesado', 'Aventura', 'Loona', 'Luis Coronel',
       'Daddy Yankee,  J. Alvarez', 'Daddy Yankee,  Natalia Jimenez',
       'Reik', 'La Arrolladora Banda el Limon de Rene Camacho',
       'Banda Sinaloense MS de Sergio Lizarraga',
       'Alejandro Fernandez ,  Christina Aguilera', 'Xtreme', 'Santana',
       'Vicente Fernandez', 'Enrique Iglesias,  Romeo Santos',
       'L

In [19]:
#Ahora separamos el artista principal (source) de los colaboradores (target).
weekly[['Principal','Collaborator']] = weekly['Artist'].str.split(',', 1, expand=True)

#Eliminamos la columna 'Artist'.
weekly = weekly.drop('Artist', axis = 1)

#Reemplazamos valores 'None' por 'Na'.
weekly.fillna('', inplace = True)

#Reordenamos columnas
column_names = ['Year', 'Month', 'Day', 'Rank', 'Title', 'Principal', 'Collaborator', 'Peak_position', 'Weeks_in_chart']
weekly = weekly.reindex(columns = column_names)

#Vemos las primeras 10 observaciones
weekly.head(10)

,Year,Month,Day,Rank,Title,Principal,Collaborator,Peak_position,Weeks_in_chart
0,2013,04,20,1,Hips Don't Lie,Shakira,Wyclef Jean,1,1
1,2013,04,20,2,Waka Waka (Esto Es Africa),Shakira,Freshlyground,2,1
2,2013,04,20,3,Danza Kuduro,Don Omar,Lucenzo,3,1
3,2013,04,20,4,Ai Se Eu Te Pego,Michel Telo,,4,1
4,2013,04,20,5,Loca,Shakira,El Cata,5,1
5,2013,04,20,6,Loba,Shakira,,6,1
6,2013,04,20,7,Promise,Romeo Santos,Usher,7,1
7,2013,04,20,8,"Suerte [Whenever, Wherever]",Shakira,,8,1
8,2013,04,20,9,En La Obscuridad,Belinda,,9,1
9,2013,04,20,10,Llevame Contigo,Romeo Santos,,10,1


In [20]:
#Hacemos explode de la variable 'Collaborator' para obtener un artista por renglón.
weekly = weekly.assign(Collaborator = weekly['Collaborator'].str.split(', ')).explode('Collaborator')
weekly = weekly.reset_index(drop = True)

#Eliminams espacios en blanco antes de cada nombre de artista en la variable 'Collaborator'
weekly['Collaborator'] = weekly['Collaborator'].str.lstrip()

weekly

,Year,Month,Day,Rank,Title,Principal,Collaborator,Peak_position,Weeks_in_chart
0,2013,04,20,1,Hips Don't Lie,Shakira,Wyclef Jean,1,1
1,2013,04,20,2,Waka Waka (Esto Es Africa),Shakira,Freshlyground,2,1
2,2013,04,20,3,Danza Kuduro,Don Omar,Lucenzo,3,1
3,2013,04,20,4,Ai Se Eu Te Pego,Michel Telo,,4,1
4,2013,04,20,5,Loca,Shakira,El Cata,5,1
...,...,...,...,...,...,...,...,...,...
13485,2020,12,26,22,La Toxica,Farruko,,6,8
13486,2020,12,26,23,Si Veo A Tu Mama,Bad Bunny,,1,40
13487,2020,12,26,24,Maldita Pobreza,Bad Bunny,,12,3
13488,2020,12,26,25,Safaera,Bad Bunny,Jowell-Randy,2,40


### II.3 Exportamos a un csv

In [24]:
#El path se estableció en la sección I.3
weekly.to_csv('weekly_charts.csv', index = False)